In [28]:
import pandas as pd
import numpy as np
import re

In [18]:
data = pd.read_json(r'C:\Users\caio-\Documents\Projetos\scrapy_cel\data\data.json')
data.head()

,title,price
0,Smartphone Samsung Galaxy S22 128gb 8gb Ram | ...,2.379
1,Smartphone Samsung Galaxy S22 128gb 8gb Ram | ...,2.379
2,Blackberry Bold 9780 256 Mb Black 512 Mb Ram,78.000
3,Lemon Sensi 3g Lm-754 3g Para Antena Rural,53.000
4,Moto G30 Dark Prism 128gb 4gb Ram - Estilo Em ...,799.000


In [23]:
data['brand'] = np.select(
    [
        data['title'].str.contains('Samsung', na=False),
        data['title'].str.contains('Moto|Motorola', na=False),
        data['title'].str.contains('Lenovo', na=False),
        data['title'].str.contains('Blackberry', na=False),
        data['title'].str.contains('LG|Lg|lg', na=False),
        data['title'].str.contains('IPHONE|iPhone|IPhone', na=False)
    ],
    [
        'Samsung',
        'Motorola',
        'Lenovo',
        'Blackberry',
        'LG',
        'iPhone'
    ],
    default=''
)

In [39]:
data.head(25)

,title,price,brand,model
0,Smartphone Samsung Galaxy S22 128gb 8gb Ram | ...,2.379,Samsung,Galaxy S22 128
1,Smartphone Samsung Galaxy S22 128gb 8gb Ram | ...,2.379,Samsung,Galaxy S22 128
2,Blackberry Bold 9780 256 Mb Black 512 Mb Ram,78.000,Blackberry,Modelo desconhecido
3,Lemon Sensi 3g Lm-754 3g Para Antena Rural,53.000,,Modelo desconhecido
4,Moto G30 Dark Prism 128gb 4gb Ram - Estilo Em ...,799.000,Motorola,Moto G30
5,Telefone Celular Samsung S10e 128 Gb 6gb Ram S...,1.089,Samsung,Modelo desconhecido
6,Motorola Moto E7 32gb,529.000,Motorola,Moto E7
7,Smartphone Samsung Galaxy J5 Quad Core 16gb/1g...,259.000,Samsung,Galaxy J5
8,Smartphone Android Simple Barato C/ Whatsapp A...,149.000,,Modelo desconhecido
9,Smartphone Android Simple Barato C/ Whatsapp A...,159.000,,Modelo desconhecido


In [38]:
patterns = {
    'Samsung': r'Galaxy \w+\s?\d*',
    'Motorola': r'Moto \w+',
    'Lenovo': r'Lenovo \w+',
    'Blackberry': r'Blackberry\d+',
    'LG': r'(G\d+ \w+)|(K\d+ \w+)',
    'iPhone': r'iPhone \d+'
}

def extract_model(title):
    for brand, pattern in patterns.items():
        match = re.search(pattern, title)
        if match:
            return match.group(0)
    return 'Modelo desconhecido'

# Aplicando a função ao DataFrame
data['model'] = data['title'].apply(extract_model)